In [1]:
import pandas as pd

input_dir = '/data/arpit.goel/16_WDInactive/'
input_file = 'get_spend_2_sel.csv'
df = pd.read_csv(input_dir+input_file,delimiter='|')



In [42]:
df_test_raw = df[df['CTL_GRP_IND']==0]
df_test = df_test_raw.sample(n=1000000)
df_test = df_test[df_test['coupon_depth'].fillna(0)>-0.035]
df_test.columns.tolist()



['XTRA_CARD_NBR',
 'HH_ID',
 'MAILER_ID',
 'CTL_GRP_IND',
 'REDEEM_ACTL_DT',
 'segment_now',
 'segment_pre6',
 'segment_pre12',
 'tenure',
 'pred_tgt_DYN_pre3',
 'pred_tgt_DYN_now',
 'WTS_CLUSTER_201601',
 'timediff',
 'coupon_depth',
 'FS_SCAN_AMT_PRE',
 'FS_SCAN_AMT_POST',
 'RED_AMT',
 'COUPON_DEPTH_BUCKET',
 'DYN_SCORE_BINS_PRE3',
 'COUNT_CARDS',
 'RDM_IND',
 'COUNT_RDM',
 'COUPON_USED_FLAG']

In [43]:
df_test1  = pd.concat([df_test,pd.get_dummies(df_test['segment_pre6'],prefix='SEGMENT_PRE6')],axis=1)
df_test2  = pd.concat([df_test1,pd.get_dummies(df_test['segment_pre12'],prefix='SEGMENT_PRE12')],axis=1)
df_test3  = pd.concat([df_test2,pd.get_dummies(df_test['WTS_CLUSTER_201601'],prefix='WTS_CLUSTER')],axis=1)


In [58]:
df_test3['SEGMENT_PRE6_TOP4'] = df_test3['segment_pre6'].map(lambda x: 1 if x in ['ECE','CRX','FFL','YBE'] else 0)
df_test3['SEGMENT_PRE12_TOP4'] = df_test3['segment_pre12'].map(lambda x: 1 if x in ['ECE','CRX','FFL','YBE'] else 0)
df_test3['SEGMENT_PRE6_TOP7'] = df_test3['segment_pre6'].map(lambda x: 1 if x in ['ECE','CRX','FFL','YBE','In&Out','ABB','CSS'] else 0)
df_test3['SEGMENT_PRE12_TOP7'] = df_test3['segment_pre12'].map(lambda x: 1 if x in ['ECE','CRX','FFL','YBE','In&Out','ABB','CSS'] else 0)

df_test3['SEGMENT_PRE6_BOTTOM6'] = df_test3['segment_pre6'].map(lambda x: 1 if x in ['One and Done','Unengaged','ECS','In&Out','ABB','CSS'] else 0)
df_test3['SEGMENT_PRE12_BOTTOM6'] = df_test3['segment_pre12'].map(lambda x: 1 if x in ['One and Done','Unengaged','ECS','In&Out','ABB','CSS'] else 0)
df_test3['SEGMENT_PRE6_BOTTOM3'] = df_test3['segment_pre6'].map(lambda x: 1 if x in ['One and Done','Unengaged','ECS'] else 0)
df_test3['SEGMENT_PRE12_BOTTOM3'] = df_test3['segment_pre12'].map(lambda x: 1 if x in ['One and Done','Unengaged','ECS'] else 0)

df_test3['WTS_PERS'] = df_test3['WTS_CLUSTER_201601'].fillna('00').map(lambda x: 1 if x[:2] in ['01','02','03']  else 0)
df_test3['WTS_MASS'] = df_test3['WTS_CLUSTER_201601'].fillna('00').map(lambda x: 1 if x[:2] in ['04','05','06','07','08']  else 0)

df_test3['REDEEM_IND'] = (df_test3['RED_AMT'].fillna(0) < 0)

df_test3['COUPON_USED_FLAG'] = (df_test3['coupon_depth'].fillna(0) < 0).astype(np.int64)
df_test3['DYN_SCORE_BINS_PRE3'] = (df_test3['pred_tgt_DYN_pre3'].fillna(0) < 0.01).astype(np.int64)


In [68]:
'''
# 'tenure',
# 'coupon_depth',                                                   \
 'SEGMENT_PRE12_ABB',                                              \
 'SEGMENT_PRE12_CRX',                                              \
 'SEGMENT_PRE12_CSS',                                              \
 'SEGMENT_PRE12_ECE',                                              \
 'SEGMENT_PRE12_ECS',                                              \
 'SEGMENT_PRE12_FFL',                                              \
 'SEGMENT_PRE12_In&Out',                                           \
 'SEGMENT_PRE12_One and Done',                                     \
 'SEGMENT_PRE12_Unengaged',                                        \
 'SEGMENT_PRE12_YBE',                                              \
 'SEGMENT_PRE12_TOP4',                                             \
 'SEGMENT_PRE12_TOP7',                                             \
 'SEGMENT_PRE12_BOTTOM3',                                          \
 'SEGMENT_PRE12_BOTTOM6',                                          \
 'SEGMENT_PRE6_ECS',                                               \
 'pred_tgt_DYN_pre3',                                              \
 'SEGMENT_PRE6_ABB',                                               \
 'SEGMENT_PRE6_CRX',                                               \
 'SEGMENT_PRE6_CSS',                                               \
 'SEGMENT_PRE6_ECE',                                               \
 'SEGMENT_PRE6_FFL',                                               \
 'SEGMENT_PRE6_In&Out',                                            \
 'SEGMENT_PRE6_One and Done',                                      \
 'SEGMENT_PRE6_Unengaged',                                         \
 'SEGMENT_PRE6_YBE',                                               \
 'WTS_CLUSTER_01_Personalized(EM)',                                \
 'WTS_CLUSTER_02_All ways to save mass+pers (full store FS+Rx)',   \
 'WTS_CLUSTER_03_Personalized (TCM)',                              \
 'WTS_CLUSTER_04_Mass Savers',                                     \
 'WTS_CLUSTER_05_Mass, PHR(RX)',                                   \
 'WTS_CLUSTER_06_Mass, All Ebs',                                   \
 'WTS_CLUSTER_07_Mass, Convenience(QEB)',                          \
 'WTS_CLUSTER_08_Mass, CEB lovers',                                \
 'SEGMENT_PRE6_TOP4',                                              \
 'SEGMENT_PRE6_TOP7',                                              \
 'SEGMENT_PRE6_BOTTOM3',                                           \
 'SEGMENT_PRE6_BOTTOM6',                                           \
 
'''
inputcolumns = [                                                         \
 'COUPON_USED_FLAG',                                                \
 'DYN_SCORE_BINS_PRE3',                                             \
 'WTS_PERS',                                                       \
 'WTS_MASS'                         
               ]

target = 'REDEEM_IND'
print inputcolumns

['COUPON_USED_FLAG', 'DYN_SCORE_BINS_PRE3', 'WTS_PERS', 'WTS_MASS']


In [69]:
from sklearn import tree

clf = tree.DecisionTreeRegressor(max_depth=4,min_samples_split=10000, min_samples_leaf=5000,random_state=123)
clf = clf.fit(df_test3[inputcolumns].fillna(0), df_test3[target])

tree.export_graphviz(clf, out_file=input_dir+'tree.dot',feature_names=inputcolumns,\
                         filled=True, rounded=True,special_characters=True)  


In [47]:
print df_test3.shape

(822153, 69)


In [16]:
df_test.columns.tolist()

['XTRA_CARD_NBR',
 'HH_ID',
 'MAILER_ID',
 'CTL_GRP_IND',
 'REDEEM_ACTL_DT',
 'segment_now',
 'segment_pre6',
 'segment_pre12',
 'tenure',
 'pred_tgt_DYN_pre3',
 'pred_tgt_DYN_now',
 'WTS_CLUSTER_201601',
 'timediff',
 'coupon_depth',
 'FS_SCAN_AMT_PRE',
 'FS_SCAN_AMT_POST',
 'RED_AMT',
 'COUPON_DEPTH_BUCKET',
 'DYN_SCORE_BINS_PRE3']

In [75]:
import numpy as np
df['segment_pre6'] = df['segment_pre6'].fillna('00.Missing')
df['segment_now'] = df['segment_now'].fillna('00.Missing')
df['COUPON_USED_FLAG'] = (df['coupon_depth'].fillna(0) < 0).astype(np.int64)
df['COUPON_DEPTH_BUCKET'] = pd.cut(df['coupon_depth'].fillna(0),bins=[-10,-0.1,-0.035,0],labels=['2.>10%','1.(3.5%,10%]','0.<=3.5%'])
df['DYN_SCORE_BINS_PRE3'] = pd.cut(df['pred_tgt_DYN_pre3'].fillna(0.0001),bins=[0,0.01,2],labels=['0.<0.01','1.>=0.01'])
df['WTS_MASS'] = df['WTS_CLUSTER_201601'].fillna('00').map(lambda x: 1 if x[:2] in ['04','05','06','07','08']  else 0)
df['COUNT_CARDS'] = 1
df['RDM_IND']  = (df['RED_AMT'].fillna(0) < 0).astype(np.int64)
df['COUNT_RDM']  = (df['RED_AMT'].fillna(0) < 0).astype(np.int64)
dimensions = ['MAILER_ID','CTL_GRP_IND','WTS_MASS','COUPON_DEPTH_BUCKET','DYN_SCORE_BINS_PRE3','RDM_IND','COUPON_USED_FLAG']
metrics = ['COUNT_CARDS','COUNT_RDM', 'FS_SCAN_AMT_PRE','FS_SCAN_AMT_POST','RED_AMT']
df.groupby(dimensions)[metrics].sum().fillna(0).to_csv(input_dir+'profile1.csv')

In [73]:
dimensions = ['MAILER_ID','CTL_GRP_IND','WTS_MASS','COUPON_DEPTH_BUCKET','DYN_SCORE_BINS_PRE3','RDM_IND']
metrics = ['COUNT_CARDS','COUNT_RDM', 'FS_SCAN_AMT_PRE','FS_SCAN_AMT_POST','RED_AMT']
df[dimensions+metrics].isnull().sum()

MAILER_ID                     0
CTL_GRP_IND                   0
WTS_MASS                      0
COUPON_DEPTH_BUCKET           0
DYN_SCORE_BINS_PRE3           0
RDM_IND                       0
COUNT_CARDS                   0
COUNT_RDM                     0
FS_SCAN_AMT_PRE        20056042
FS_SCAN_AMT_POST       20056042
RED_AMT                23992991
dtype: int64

In [35]:
df[['coupon_depth','pred_tgt_DYN_pre3']].describe()

,coupon_depth,pred_tgt_DYN_pre3
count,24021135.000000,19831943.000000
mean,-0.030837,0.007840
std,0.093203,0.014742
min,-3.200000,0.003267
25%,-0.006241,0.004771
50%,0.000000,0.006186
75%,0.000000,0.007768
max,0.000000,0.352246
